In [1]:
#pragma cling add_include_path("../../include")
#pragma cling add_include_path("../feltor/inc") // Feltor path
#define THRUST_DEVICE_SYSTEM THRUST_DEVICE_SYSTEM_CPP
#include <iostream>
#include "dg/algorithm.h"

In file included from input_line_8:2:
In file included from ../../include/dg/algorithm.h:8:
../../include/dg/backend/config.h:20:9: warning: NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead! [-W#pragma-messages]
#pragma message( "NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead!")
        ^
In file included from input_line_8:2:
In file included from ../../include/dg/algorithm.h:11:
In file included from ../../include/dg/topology/split_and_join.h:4:
In file included from ../../include/dg/backend/blas1_dispatch_shared.h:12:
In file included from ../../include/dg/backend/blas1_serial.h:6:
In file included from ../../include/dg/backend/exblas/exdot_serial.h:25:
In file included from ../../include/dg/backend/exblas/accumulate.h:19:
../../include/dg/backend/exblas/config.h:31:9: warning: WARNING: Instruction set below SSE4.1! Deactivating vectorization!
      [-W#pragma-messages]
#pragma message("WARNING: Instruction set below SSE4.1! Deactivating vectorization!")
 

# Solvers in Feltor

## A first example: inverting elliptic equations

With grids and derivatives available we are now finally in the position
to deal with high level numerical algorithms. One example of such
an algorithm is the discretization and inversion of elliptic equations, for example Poison's equation.
Look at the following code, which inverts a Laplacian


In [2]:
#include <iostream>
#include "dg/algorithm.h"

In [3]:
const double lx = 2.*M_PI;
const double ly = 2.*M_PI;
unsigned n = 3, Nx = 10, Ny = 10;
const double eps = 1e-4;

In [4]:
double fct(double x, double y){
    return sin(y) * sin(x);
}

In [5]:
double laplace_fct( double x, double y) {
    return 2 * sin(y) * sin(x);
}

In [6]:
double initial( double x, double y) {
    return sin(0);
}


In [7]:
// The numbers in parentheses are our suggestions ...
//std::cout << "Type n (3), Nx (20) and Ny (20)! \n";
//std::cin >> n >> Nx >> Ny;
std::cout << "Computing on the Grid " <<n<<" x "<<Nx<<" x "<<Ny <<"\n";
dg::CartesianGrid2d grid( 0, lx, 0, ly, n, Nx, Ny, dg::DIR, dg::PER);
std::cout<<"Evaluate initial guess for iterative scheme\n";
dg::DVec x = dg::evaluate( initial, grid);
// create volume and inverse volume on previously defined grid
const dg::DVec vol2d = dg::create::volume( grid);

// Create negative, unnormalized, positive definite Laplacian
dg::Elliptic<dg::CartesianGrid2d, dg::DMatrix, dg::DVec> laplaceM( grid);
// allocate memory in conjugate gradient
unsigned max_iter = n*n*Nx*Ny;
dg::PCG<dg::DVec > pcg( x, max_iter);

// Evaluate right hand side and solution on the grid
dg::DVec b = dg::evaluate ( laplace_fct, grid);
const dg::DVec solution = dg::evaluate ( fct, grid);
dg::Timer t;
t.tic();
// we do not use a preconditioner in solution method
unsigned number = pcg.solve( laplaceM, x, b, 1., vol2d, eps);
t.toc();
std::cout << "Number of pcg iterations "<< number <<"\n";
std::cout << "For a precision of "<< eps<<"\n";
// clang as script is unfortunately not very fast so don't make the numbers too big
std::cout << "Took "<< t.diff()<<"s\n";

Computing on the Grid 3 x 10 x 10
Evaluate initial guess for iterative scheme
Number of pcg iterations 15
For a precision of 0.0001
Took 1.80696s


In [8]:
//compute error
dg::DVec error( solution);
dg::blas1::axpby( 1.,x,-1.,error);

dg::DVec lap_x(x), residuum( b);
dg::blas2::symv(  laplaceM, x, lap_x);
dg::blas1::axpby( 1., lap_x, -1., residuum);

//global relative error in L2 norm is O(h^n)
double result;
result = sqrt(dg::blas2::dot( x, vol2d, x));
std::cout << "L2 Norm of x is               " << result << std::endl;
result = sqrt(dg::blas2::dot(solution, vol2d , solution));
std::cout << "L2 Norm of Solution is        " << result << std::endl;
result = sqrt(dg::blas2::dot(error, vol2d , error));
std::cout << "L2 Norm of Error is           " << result << std::endl;
result = sqrt(dg::blas2::dot( residuum, vol2d, residuum));
std::cout << "L2 Norm of Residuum is        " << result << std::endl;

L2 Norm of x is               3.14152
L2 Norm of Solution is        3.14159
L2 Norm of Error is           0.00348843
L2 Norm of Residuum is        0.000467984


This program executes on the device as is evident from the use of
 `dg::DVec`, a typedef for `thrust::device_vector<double>`, and `dg::DMatrix`.
 The new class that we encounter in this program is `dg::Elliptic`, a
 level 4 class. What this class does is to create and store the `dx` and `dy` matrices from the given grid. Furthermore, it allocates some
 internal workspace and defines a member function that uses `dg::blas1` and `dg::blas2` functions to discretize the Laplacian.
 The class depends on three template parameters,
 the geometry, the matrix and the vector class. The matrix and the
 vector class must fit together, that is, they must be useable in
 a `dg::blas2::symv` function.  Note, that the `dg::Elliptic` class
 acts as a matrix itself. This becomes clear in the line `dg::blas2::symv( laplaceM, x, lap_x)`. Note that the `M` in `laplaceM`
 reminds us that `dg::Elliptic` discretizes the negative Laplacian in order to
 generate a postive definite operator.

 The geometry `dg::CartesianGrid2d` defines a two-dimensional Euclidean metric
 tensor. Recall, that in general the Laplacian depends on metric coefficients.

Since the `dg::Elliptic` is a matrix, we can now use it in a
conjugate gradient solver. The `dg::CG` is a Level 2 class and thus
only depends on the vector class that we use. Recall here that the
conjugate gradient algorithm can be implemented with vector addition `dg::blas1::axpby`,
a scalar product (`dg::blas1::dot`) and a matrix-vector multiplication (`dg::blas2::symv`) alone.

In the above example we inverted the simple Laplacian. However, recall here
that a conjugate gradient algorithm works for **any** matrix that is symmetric
and positive definite. We provide a wide selection of already implemented
matrix types in our library but you can also provide your very own matrix class
in the above program. In order for this to work your own matrix class must be a functor/lambda with the signature `void operator()(const container&,container&)` where `container` is a placeholder for the vector class you use.


## Operators and Solvers
Feltor has a variety of solvers available that all solve the standard form 
\begin{align}\label{eq:standard}
    f(x) = b
\end{align}
for either linear or non-linear function $f$.
These are for example Conjugate Gradient PCG, LGMRES, ChebyshevIteration, BICGSABl and the nonlinear fixed point iteration and its generalization, the AndersonAcceleration. Furthermore, we have the multigrid family of solvers, in
particular our nested iterations solver.
These solvers have in common that they are called in some
variation of

```{code-block} cpp
Operator op(...); // some operator
Solver solver( copyable, max_iter);
Vector x, b;
Vector weights;
double rtol = 1e-8, atol = rtol;
solver.solve( op, x, b, weights, rtol, atol, more_params...);
```

Here, the weights define the scalar product
\begin{align}
   \langle x, y \rangle := \sum_i x_i w_i y_i
\end{align}
in which the error norm in particular and the scalar product
is defined. 

### The `dg::apply` function
All solvers also have in common that their first parameter
in the solve method is the operator that computes the function $f(x)$. It needs to be either a functor or a matrix that is the function


```{code-block} cpp
    dg::apply( op, x,y); // compute y = f(x)
    dg::blas2::symv( op, x,y );//completely equivalent
```

will first check if the type of the operator has the `dg::TensorTraits<Operator>` specialized.
If yes, it will assume the operator is a matrix and
apply one of our specialized implementations. If not, then
it will try to call

```{code-block} cpp
op(x,y); // compute y = f(x)
```

i.e. the apply call is equivalent to a functor call.
This observation is important to understand the following.
### The operators as functors
Feltor has an assortment of already made Operators available that have special implementations of the symv function available, for example the
`dg::Elliptic` or `dg::Helmholtz` classes. Sometimes however, you need special operators, for example an implicit timestepper wants you to solve $x + \alpha I(x,t) = b$. Assuming you implement $I(x,t)$, then this is not in the standard form Eq.~\eqref{eq:standard} that the solvers expect.
In order to make a new operator usable in the solve methods you have two options
#### Overload operator()) 
You could implement a new class and overload

```{code-block} cpp
struct Operator
{
    template<class ContainerType0, class ContainerType1>
    void operator()( const ContainerType0& x, ContainerType1& y);
};
```

to make the class usable in the `dg::apply`
function. This is useful for complicated functors that hold
internal state. It is less useful for very small classes, where the parenthesis operator only consists of a line or two and when the operator is basically only 
an adaptor of another type.
#### Lambda functions
In C++14 lambda functions are a very powerful tool. 
They are basically functors that
can be generated in a very concise and short way. 
```{seealso}
If you are unfamiliar with lambdas a good watch on youtube is for example [Lambdas from Scratch - Arthur O'Dwye](https://www.youtube.com/watch?v=3jCOwajNch0)
```
For the above example of the
timestepper we could for example write

```cpp
Implicit imp(...);
double alpha  = ..., time = ...;
// We want an operator that computes x + a I(x,t) for given I
auto functorToInvert = [ a = alpha, t = time, &im = imp]
    (const auto& x, auto& y)
{
    im(t,x,y); // we assume that im is available
    dg::blas1::axpby( 1., x, a, y);
} 
```


Several things are noteworthy here

  - Our lambda is a template or a "generic lambda" through the use of "auto" in the interface
  - Our lambda uses "init capture", a feature from C++14, which could be shortened in our case but here makes the code more verbose. Also note that we caught the implicit object by reference (in order to avoid a deep copy but we should beware "dangling references") while it captures alpha and time by value.

Note that the lambda now has the correct interface to be used in a solver.

```cpp
dg::PCG<Vector> pcg(...);
int number = pcg.solve( functorToInvert, x, b,
                        preconditioner, weights, 1e-10);
```

### Preconditioning or the inverse operators
The linear Feltor solvers all can be used with preconditioners. From an
interface point of view, the Preconditioners have the same conditions 
as the operator itself, i.e. they must be usable as functors

```cpp
Precond p(...);
dg::apply( p, x,y); // computes y = P(x)
```


This example trivially works if `using Precond = Vector` is a 
vector, i.e. a diagonal preconditioner. But what if we want to use a more involved preconditioner?

Again, there are two options. The first option is to write an entirely new class
and overload the function call operator as described in Section~\ref{sec:overload}.
The second one is to combine existing solvers to construct a new one with the use of lambdas as described in Section~\ref{sec:lambda}

A preconditioner is essentially an approximate inverse. Here, we show
how to use a lambda to combine an elliptic operator with a Chebyshev solver that applies 10 iterations as a preconditioner that can be used in PCG (solves $-\Delta \Phi = \rho $)

```cpp
// The operator that we want to solve
dg::Elliptic<...> pol( grid);
pol.set_chi( chi);
//estimate EigenValue
dg::EVE<Vector> eve( copyable);
double eps_ev = 1e-2, ev;
// compute largest Eigenvalue in a few iterations
counter = eve.solve( pol, x, b, 1., pol.weights(), ev, eps_ev);
// Here we combine the Elliptic operator with the Chebyshev solver
dg::ChebyshevIterations<Vector> cheby( x);
auto precond = [nu=10, ev, &pol, &cheby](const auto& y, auto& x)
        {
            cheby.solve( pol, x, y, 1., ev/100., ev*1.1, nu+1, true);
        };
// Now we can call the solver
dg::PCG< Vector > pcg( x, 1000);
pcg.solve( pol, x, y, precond, pol.weights(), eps, 1, 1);
```

## The multigrid solvers
In order to use a multigrid solver like nested iteration we first need to
be able to construct the operators on various grids

```cpp
unsigned stages = 3;
Geometry grid(...);
dg::NestedGrids<Geometry, Matrix, Container> nested( grid, stages);
```

The nested grids provide the functionality to project and interpolate between
various grids and also provides a workspace for the solvee function.
We can use this to construct 

```cpp
std::vector<Container> multi_chi = nested.project( chi);
std::vector<dg::PCG<Container> > multi_pcg( stages);
std::vector<dg::Elliptic<Geometry, Matrix, Container> >
    multi_pol( stages);
for(unsigned u=0; u<stages; u++)
{
    multi_pol[u].construct( nested.grid(u), dg::centered);
    multi_pol[u].set_chi( multi_chi[u]);
    multi_pcg[u].construct( multi_chi[u], 1000);
}
```

We use the nested grids to construct an operator and a solver on each grid.
Now we are ready to construct the center-piece of our multigrid solver.
We need to construct an inverse operator (that is an operator combined with a solver) on each grid and store it

```cpp
std::vector<std::function<void( const dg::DVec&, dg::DVec&)> > 
    multi_inv_pol(stages);
for(unsigned u=0; u<stages; u++)
{
    multi_inv_pol[u] = [=, &pcg = multi_pcg[u], &pol = multi_pol[u]) ](
        const auto& y, auto& x)
    {
#ifdef MPI_VERSION
        int rank;
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);
#endif //MPI
        dg::Timer t;
        t.tic();
        number = pcg.solve( pol, x, y, pol.precond(), pol.weights(),
                eps, 1., 1 );
        t.toc();
        DG_RANK0 std::cout << "# Nested iterations stage: " << u << ", iter: " << number << ", took "<<t.diff()<<"s\n";
    };
}
```

Note again how we capture the solver and operator by reference in the lambda. This 
is very similar to how we constructed the preconditioner in section \ref{sec:precond}. Also note that we store the lambda in a vector of `std::function< ...>`, which erases the type of the lambda. Therefore, it is possible to use different kind of solvers for each stage.
Finally, note that we use the lambda body to also provide some ease of living
benchmark functionality.
The final step is simply to call the appropriate function

```cpp
nested_iterations( multi_pol, x, b, multi_inv_pol, nested);
```